In [1]:
import json
import logging
import os
from pathlib import Path

from wattile.buildings_processing import prep_for_rnn
from wattile.data_reading import read_dataset_from_file
from wattile.entry_point import init_logging
from wattile.models import AlfaModel

PROJECT_DIRECTORY = Path().resolve().parents[1]
logger = logging.getLogger(str(os.getpid()))

PROJECT_DIRECTORY = C:\Users\JKIM4\Anaconda3\envs\wattile\Lib\site-packages\wattile


In [2]:
"""
define parametric space
"""
parametric_space = {
    1: {
        "building": "Synthetic Site",
        "targets": [
            "Synthetic Site Electricity Main Total Power"
            # add more targets if necessary
        ],
        "dir_data": "tests/data/Synthetic Site",
    }
    # add more entries if necessary
}

In [3]:
"""
run model training for the entire sample space
"""
for entry in parametric_space:

    # set building name
    building = parametric_space[entry]["building"]

    # set data directory
    data_dir = parametric_space[entry]["dir_data"]

    for target in parametric_space[entry]["targets"]:

        logger.info(
            "training for building = {} and target = {}".format(building, target)
        )

        # read configs file for training
        with open(
            PROJECT_DIRECTORY / "wattile" / "configs" / "configs.json", "r"
        ) as f:
            configs = json.load(f)

        # override configs params based on the sample space information
        configs["data_input"]["data_dir"] = str(PROJECT_DIRECTORY / data_dir)
        configs["data_input"]["data_config"] = "{} Config.json".format(building)
        configs["data_input"]["target_var"] = target
        exp_dir = PROJECT_DIRECTORY / building / target
        configs["data_output"]["exp_dir"] = str(exp_dir)

        # create results folder
        init_logging(local_results_dir=configs["data_output"]["exp_dir"])

        # read data
        data = read_dataset_from_file(configs)

        # prepare data for training
        train_df, val_df = prep_for_rnn(configs, data)

        # train model
        model = AlfaModel(configs)
        results = model.main(train_df, val_df)

        logger.info(
            "validating for building = {} and target = {}".format(building, target)
        )

        # read configs file for validation
        with open(exp_dir / "configs.json", "r") as f:
            configs = json.load(f)

        # reset config param for validation
        configs["data_output"]["plot_comparison_portion_start"] = 0.0
        configs["data_output"]["plot_comparison_portion_end"] = 1.0
        configs["learning_algorithm"]["use_case"] = "validation"

        # prepare data for training
        train_df, val_df = prep_for_rnn(configs, data)

        # train model
        model = AlfaModel(configs)
        results = model.main(train_df, val_df)



Logging to: C:\Users\JKIM4\Documents\GitHub\intelligentcampus-pred-analytics\Synthetic Site\Synthetic Site Electricity Main Total Power\output.out, PID: 9264
saving timeseries comparison in C:\Users\JKIM4\Documents\GitHub\intelligentcampus-pred-analytics\Synthetic Site\Synthetic Site Electricity Main Total Power/Vis_TimeseriesComparisons.svg
saving timeseries comparison in C:\Users\JKIM4\Documents\GitHub\intelligentcampus-pred-analytics\Synthetic Site\Synthetic Site Electricity Main Total Power/Vis_TimeseriesComparisons.svg
